Part 1. Charts
1. Find/Select the dataset for creating charts dashboard*
2. Create 3 charts using Excel, Power BI, KNIME or Python
3. Specify the business need and design justification for each chart using the template


In [2]:
import pandas as pd
import altair as alt


In [3]:
file_path = "../data/Lemonade_formatted.xlsx"

df = pd.read_excel(file_path)

display(df.head())

# Display column types and non-null counts
print("\nColumn Info:")
df.info()

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Date,Month,Day,Temperature,Rainfall,Flyers,Price,Sales,Revenue
0,2017-01-01,January,Sunday,27.0,2.00,15.0,0.3,10.0,3.0
1,2017-01-02,January,Monday,28.9,1.33,15.0,0.3,13.0,3.9
2,2017-01-03,January,Tuesday,34.5,1.33,27.0,0.3,15.0,4.5
3,2017-01-04,January,Wednesday,44.1,1.05,28.0,0.3,17.0,5.1
4,2017-01-05,January,Thursday,42.4,1.00,33.0,0.3,18.0,5.4



Column Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         365 non-null    datetime64[ns]
 1   Month        365 non-null    object        
 2   Day          365 non-null    object        
 3   Temperature  365 non-null    float64       
 4   Rainfall     365 non-null    float64       
 5   Flyers       365 non-null    float64       
 6   Price        365 non-null    float64       
 7   Sales        365 non-null    float64       
 8   Revenue      366 non-null    float64       
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 25.9+ KB


In [4]:
# Calculate correlation
correlation = df['Flyers'].corr(df['Revenue'])

print(f"The correlation between revenue and flyers is: {correlation:.2f}")

The correlation between revenue and flyers is: 0.68


In [5]:

chart = alt.Chart(df).mark_circle(opacity=0.7).encode(
    x=alt.X('Flyers', title='Number of flyers'),
    y=alt.Y('Revenue', title='Revenue (USD)'),
    tooltip=['Date', 'Flyers', 'Revenue', 'Temperature', 'Sales']
).properties(
    title='How flyer distribution affects sales'
).interactive()

chart

alt.Chart(...)

In [6]:
chart.save('revenue_vs_flyers.png')
chart.save('revenue_vs_flyers.svg')

In [7]:


# A bar chart with monthly aggregated revenue
monthly_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Date',
            timeUnit='yearmonth',
            title='Month',
            axis=alt.Axis(format='%b %Y')
           ),


    y=alt.Y('Revenue',
            aggregate='sum',
            title='Revenue (USD)'
           ),

    # Tooltips on hover
    tooltip=[
        alt.Tooltip('Date', timeUnit='yearmonth', format='%b %Y', title='Month'),
        alt.Tooltip('Revenue', aggregate='sum', title='Total Revenue', format='$,.2f')
    ]
).properties(
    title='Revenue per month (2017)'
).interactive()

monthly_chart.save('monthly_revenue.png')
monthly_chart.save('monthly_revenue.svg')
monthly_chart

alt.Chart(...)

In [8]:
#Total Revenue by Day of Week in a pie chart

daily_revenue = df.groupby('Day')['Revenue'].sum().reset_index()

week_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

daily_revenue['percentage'] = daily_revenue['Revenue'] / daily_revenue['Revenue'].sum()


base = alt.Chart(daily_revenue).encode(
    theta=alt.Theta("Revenue", stack=True)
)

pie = base.mark_arc(outerRadius=120).encode(
    color=alt.Color("Day", sort=week_order),
    order=alt.Order("Revenue", sort="descending"),

    tooltip=[
        "Day",
        alt.Tooltip("Revenue", format="$,.2f", title="Total Revenue"),
        alt.Tooltip("percentage", format=".1%", title="Percentage")
    ]
).properties(
    title="Revenue distribution by day of the week (2017)"
)

text = base.mark_text(radius=140).encode(
    text=alt.Text("percentage", format=".1%"),
    order=alt.Order("Revenue", sort="descending"),
    color=alt.value("black")
)

revenue_pie_chart = pie + text

revenue_pie_chart.save('revenue_by_weekday_pie.png')
revenue_pie_chart.save('revenue_by_weekday_pie.svg')

revenue_pie_chart

alt.LayerChart(...)

In [9]:
# Variable Weather scatter plot

three_var_chart = alt.Chart(df).mark_circle(opacity=0.8, stroke='black', strokeWidth=0.2).encode(
    x=alt.X('Temperature', title='Temperature (°F)'),
    y=alt.Y('Flyers', title='Number of flyers'),

    color=alt.Color('Rainfall',
                    scale=alt.Scale(scheme='plasma'),
                    title='Rainfall (inches)'),

    tooltip=[
        'Date',
        'Temperature',
        'Rainfall',
        'Flyers'
    ]
).properties(
    title='How temperature and rainfall affect Flyer distribution'
).interactive()

three_var_chart.save('temp_rain_flyers.png')
three_var_chart.save('temp_rain_flyers.svg')

three_var_chart

alt.Chart(...)

In [10]:
# Correlation calculations
print("--- Correlation Analysis ---")
corr_temp_flyers = df['Temperature'].corr(df['Flyers'])
print(f"Temperature vs. Flyers: {corr_temp_flyers:.2f}")

corr_rain_flyers = df['Rainfall'].corr(df['Flyers'])
print(f"Rainfall vs. Flyers:    {corr_rain_flyers:.2f}")

corr_temp_rain = df['Temperature'].corr(df['Rainfall'])
print(f"Temperature vs. Rainfall: {corr_temp_rain:.2f}")
print("----------------------------\n")

--- Correlation Analysis ---
Temperature vs. Flyers: 0.80
Rainfall vs. Flyers:    -0.75
Temperature vs. Rainfall: -0.90
----------------------------

